In [216]:
import pandas as pd
pd.options.display.max_rows=20

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
%matplotlib inline

import string 
import warnings
import time,datetime
import random
from __future__ import print_function

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls
plotly.offline.init_notebook_mode(connected=True)

from IPython.display import display, HTML
from ipywidgets import interactive
from ipywidgets import interact, widgets, fixed
from ipywidgets import interact_manual
from pivottablejs import pivot_ui

warnings.simplefilter("ignore")

#==========Flags=============

verbose=True

#============================


if verbose: print('initalize notebook...')

if verbose: print('reading current deflators ...')
MADWiki   = pd.ExcelFile('OMB-Hist_Tab10-1.xlsx')
gdpf = pd.read_excel(MADWiki)
pgdp = gdpf[gdpf.columns[1]]                 #pgdp deflator

xlsx   ='OCO.xlsx'                      #read OCO
if verbose: print('reading '+xlsx+' ...')
wb     = pd.ExcelFile(xlsx)
sheets = wb.sheet_names
oco= wb.parse('OCO_BA') 
if verbose:print('processing OCO data ...')
oco=oco.head(9)
oco.set_index(oco.columns[0],inplace=True)
oco=oco.T
oco.columns=oco.iloc[0].ravel()
oco=oco.dropna(axis=1,how='all')
oco.index.name='FY'
oco.columns=['Military Personnel','O&M','Procurement','RDT&E',
             'Military Construction','Family Housing','Revolving Funds & Other','Total']
oco=oco.fillna(0,inplace=True)

xlsx   ='OMB-PBDB_FY00-FY19.xlsx'
if verbose: print('reading FYDP data from OMB Public Database '+xlsx+' (this will take a minute) ...')
wb     = pd.ExcelFile(xlsx)
sheets = wb.sheet_names
clist=[]
if verbose: print('consolidating FYDP data (this will take a minute) ...')
for sheet in sheets[:-1]:
    ws = pd.read_excel(wb, sheetname=sheet, skiprows=1)
    s=int(sheet)
    ws['PB'] = s
    clist.append(ws)
data=pd.concat(clist,axis=0,ignore_index=False)
data.set_index(['PB'],inplace=True)

#fix column names and drop junk
data.drop([u'1976',u'TQ',u'1977',u'1978',u'1979'],axis=1,inplace=True)
col=data.columns.tolist()
entry=col[:11]
fy = [int(i) for i in col[11:]]
col=entry+fy
data.columns=col

#Identify unique entries for each category in the database
Agency=np.unique(data['Agency Name'].tolist())
Bureau=np.unique(data['Bureau Name'].tolist())
Account=np.unique(data['Account Name'].tolist())
Subfunction=np.unique(data['Subfunction Title'].tolist())
PB=np.unique(data.index.tolist())
data['nonBase']=np.nan
data['Requested']=np.nan
data['Enacted']=np.nan
for b in PB:
    data['Requested'].loc[b]=data[b].loc[b]
    data['Enacted'].loc[b]=data[b-2].loc[b]

data_nom=data.copy()
data_real=data.copy()
if verbose: print('... finished')

initalize notebook...
reading current deflators ...
reading ../OCO/PB19-OCO.xlsx ...
processing OCO data ...
reading FYDP data from OMB Public Database OMB-PBDB_FY00-FY19.xlsx (this will take a minute) ...
consolidating FYDP data (this will take a minute) ...
... finished


In [263]:
MADWiki   = pd.ExcelFile('OMB-Hist_Tab10-1.xlsx')
gdpf = pd.read_excel(MADWiki,skiprows=2)
gdpf=gdpf[gdpf.columns[:3]]
gdpf.columns=['FY','GDP','pgdp']
gdpf.drop(gdpf.index[[0,1,87]],inplace=True)
gdpf=gdpf.loc[gdpf.index>42]
gdpf.FY=range(1980,2024,1)
gdpf.set_index('FY',inplace=True)
pgdp=gdpf[gdpf.columns[1]]
display()

In [217]:
def plotit(funding,Agency,Bureau,Account,Subfunction,BEA,pb,pretty=True,wdat=False):
    funds=funding[funding.columns[11:]]
    funds=funds[funds.columns[:-3]]

    funds=funds.groupby(funds.index).sum()
    pf=funds.loc[pb]/1000/1000
    pf=pf.replace(0,np.nan)
    pf=pf.div(gdpf.pgdp.div(gdpf.pgdp.loc[pb]))
    pf=pf.loc[pf.index>1979]
    pf=pf.loc[pf.index<pb+4]

    f, (ax0)= plt.subplots(1, 1, figsize=(14,9))
    width=0.8
    fs=16

    pf.plot(ax=ax0,style='-', color='royalblue', linewidth=2)
      
    ax0.set_xlabel('Fiscal Year',fontsize=fs+2)
    ax0.set_ylabel('Yearly Funding [real, $B]',fontsize=fs+3)
    
    
    ax0.tick_params(direction='in',pad=15,labelsize=fs)

    ax0.set_ylim([0, pf.max(skipna=True).max()*(1+0.05)])
    
    xmax=pf.index.max()
    xmin=pf.index.min()

    ax0.set_xlim(xmin,xmax)
    ax0.xaxis.set_ticks(np.arange(xmin, xmax, 5))
    ax0.xaxis.set_ticklabels(np.arange(xmin, xmax, 5))
    
       
    if (pretty):
        plt.show()
    else:
        plotly_fig = tls.mpl_to_plotly( f )
        plotly.offline.iplot(plotly_fig)

    if(wdat):
        wb_out ='OMB_PB_Data-query.xlsx'
        writer = pd.ExcelWriter(wb_out)
        funding.to_excel(writer,'data')
        pf.to_excel(writer,'plot')
        writer.save()

def WordSearch(funding,Category='Account Name',word=''):
    funding=funding[df[Category].str.contains(word)==True]
    pb=2019
    print('Entries associated with the word/phrase:\n',
         word,'\nin the Category:',Category,
         'for PB',pb)
    interact(plotit,funding=fixed(funding),
             Agency=np.unique(funding['Agency Name'].tolist()),
              Bureau=np.unique(funding['Bureau Name'].tolist()),
              Account=np.unique(funding['Account Name'].tolist()),
              Subfunction=np.unique(funding['Subfunction Title'].tolist()),
              BEA=np.unique(funding['BEA Category'].tolist()),
             pb=fixed(pb),
             pretty=True,wdat=False)
    
funding=data.copy()
Categories=['Agency Name',
 'Bureau Name',
 'Account Name',
 'Subfunction Title']
interact_manual(WordSearch,funding=fixed(funding),Category=Categories,word='')

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUnQ2F0ZWdvcnknLCBpbmRleD0yLCBvcHRpb25zPSgnQWdlbmN5IE5hbWUnLCAnQnVyZWF1IE5hbWUnLCAnQWNjb3XigKY=


<function __main__.WordSearch>

In [218]:
def datafoo(funding,pb,tag='Account Name',query='',write_data=False):
    
    funding[tag]=funding[tag].str.lower()    
    funding=funding[funding[tag].str.contains(query,na=False)]
    print('PB:  ',pb,
              '\nAgency Name:  ',np.unique(funding['Agency Name'].tolist()),
              '\nBureau Name:  ',np.unique(funding['Bureau Name'].tolist()),
              '\nAccount Name:  ',np.unique(funding['Account Name'].tolist()),
              '\nSubfunction Title:  ',np.unique(funding['Subfunction Title'].tolist()),
              '\nFunding Category:  ',np.unique(funding['BEA Category'].tolist()))   
    funding=funding.groupby(funding.index).sum()
    funding['fydp']=funding[funding.index.tolist()].loc[pb]
    display(funding)

    if(write_data):
        wb_out ='OMB_PB_Data-query.xlsx'
        writer = pd.ExcelWriter(wb_out)
        funding.to_excel(writer,'data')
        writer.save()
    
def functD(funding,tag,choices,query=''):
    interact(datafoo,funding=fixed(funding),
             pb=widgets.IntSlider(min=2000, max=2019, step=1, value=2012, description='PB'),
             tag=fixed(tag),query=fixed(query),pretty=True,wdat=False)

def functC(funding,tag='Account Name'):  
    funding[tag]=funding[tag].astype(str).str.lower()    
    queries=funding[tag].unique()
    queries.sort()
    interact_manual(functD,funding=fixed(funding),tag=fixed(tag),choices=queries,query='')

def functB(Category,Entry='Department of Defense-Military'):
    
    qdata=data.loc[data[Category]==Entry]
    tags=qdata.columns[:11]
    print('Pick category to query for',Category, ':\n',Entry)
    interact(functC,funding=fixed(qdata),tag=tags)
    
def functA(Category='Subfunction Title'):
    Entries=data[Category].unique()
    Entries.sort()
    interact(functB,Category=fixed(Category),Entry=Entries)
    
    
Categories=data.columns[:11].astype(str).tolist()
interact(functA,Category=Categories)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUnQ2F0ZWdvcnknLCBpbmRleD04LCBvcHRpb25zPSgnQWdlbmN5IENvZGUnLCAnQWdlbmN5IE5hbWUnLCAnQnVyZWHigKY=


<function __main__.functA>

In [219]:
def dataplot(funding,pb=2012,tag='Account Name',query='',pretty=True,wdat=False):  
    
    funding[tag]=funding[tag].str.lower()    
    funding=funding[funding[tag].str.contains(query,na=False)]
    print('PB:  ',pb,
              '\nAgency Name:  ',np.unique(funding['Agency Name'].tolist()),
              '\nBureau Name:  ',np.unique(funding['Bureau Name'].tolist()),
              '\nAccount Name:  ',np.unique(funding['Account Name'].tolist()),
              '\nSubfunction Title:  ',np.unique(funding['Subfunction Title'].tolist()),
              '\nFunding Category:  ',np.unique(funding['BEA Category'].tolist()))   
    funding=funding.groupby(funding.index).sum()
    funding['fydp']=funding[funding.index.tolist()].loc[pb]
        
    pf=funding[['fydp','Requested','Enacted']]/1000.0/1000.0
    foo = pf.copy()
    foo.reset_index(inplace=True)
    foo.PB=foo.PB-2
    foo.set_index(foo.PB, inplace=True)
    pf.Enacted=foo.Enacted
    pf=pf.replace(0,np.nan)
    pf['pgdp']=pgdp
    pf=pf[['fydp','Requested','Enacted']].div(pf.pgdp,axis=0)

    f, (ax0)= plt.subplots(1, 1, figsize=(14,9))
    width=0.8
    fs=16

    pf.plot(ax=ax0,style='-', color='royalblue', linewidth=2,
        y=['Enacted'])
    
    pf.plot(ax=ax0,style=':', color='darkolivegreen', linewidth=2,
        y=['Requested'])
    
    pf.plot(ax=ax0,style='--', color='firebrick', linewidth=1,
        y=['fydp'])
    
    ax0.set_xlabel('Fiscal Year',fontsize=fs+2)
    ax0.set_ylabel('Yearly Funding [real, $B]',fontsize=fs+3)
    
    
    ax0.tick_params(direction='in',pad=15,labelsize=fs)

    ax0.set_ylim([0, pf.max(skipna=True).max()*(1+0.05)])
    
    xmax=pf.index.max()
    xmin=pf.index.min()

    ax0.set_xlim(xmin,xmax)
    ax0.xaxis.set_ticks(np.arange(xmin, xmax, 2))
    ax0.xaxis.set_ticklabels(np.arange(xmin, xmax, 2))
    
   #ax0.set_title('Data from '+tag+' '+query, fontsize=fs+4,position=(0.8, 0.07))
    l=ax0.legend(loc='best',frameon=False)
    l.get_texts()[0].set_text('Enacted')
    l.get_texts()[1].set_text('Requested')
    l.get_texts()[2].set_text('FYDP Requested')
        
    if (pretty):
        plt.show()
    else:
        plotly_fig = tls.mpl_to_plotly( f )
        plotly.offline.iplot(plotly_fig)

    if(wdat):
        wb_out ='OMB_PB_Data-query.xlsx'
        writer = pd.ExcelWriter(wb_out)
        funding.to_excel(writer,'data')
        pf.to_excel(writer,'plot')
        writer.save()


def functD(funding,tag,choices,query=''):
    interact(dataplot,funding=fixed(funding),
             pb=widgets.IntSlider(min=2000, max=2019, step=1, value=2012, description='PB'),
             tag=fixed(tag),query=fixed(query),pretty=True,wdat=False)

def functC(funding,tag='Account Name'):  
    funding[tag]=funding[tag].astype(str).str.lower()    
    queries=funding[tag].unique()
    queries.sort()
    interact_manual(functD,funding=fixed(funding),tag=fixed(tag),choices=queries,query='')

def functB(Category,Entry='Department of Defense-Military'):
    
    qdata=data.loc[data[Category]==Entry]
    tags=qdata.columns[:11]
    print('Pick category to query for',Category, ':\n',Entry)
    interact(functC,funding=fixed(qdata),tag=tags)
    
def functA(Category='Subfunction Title'):
    Entries=data[Category].unique()
    Entries.sort()
    interact(functB,Category=fixed(Category),Entry=Entries)
    
    
Categories=data.columns[:11].astype(str).tolist()
interact(functA,Category=Categories)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUnQ2F0ZWdvcnknLCBpbmRleD04LCBvcHRpb25zPSgnQWdlbmN5IENvZGUnLCAnQWdlbmN5IE5hbWUnLCAnQnVyZWHigKY=


<function __main__.functA>

In [220]:
from IPython.display import display, HTML
HTML('''<script>
code_show=false; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')